# Step 1 - Data Engineering

The climate data for Hawaii is provided through two CSV files. Start by using Python and Pandas to inspect the content of these files and clean the data.

* Create a Jupyter Notebook file called `data_engineering.ipynb` and use this to complete all of your Data Engineering tasks.

* Use Pandas to read in the measurement and station CSV files as DataFrames.

* Inspect the data for NaNs and missing values. You must decide what to do with this data.

* Save your cleaned CSV files with the prefix `clean_`.


In [1]:
# Dependencies
import pandas as pd
import numpy as np
import csv

In [2]:
# read the provided CSV data into DataFrames
stations="hawaii_stations.csv"
weather="hawaii_measurements.csv"
stations_data=pd.read_csv(stations,encoding="iso-8859-1")
weather_data=pd.read_csv(weather,encoding="iso-8859-1")
stations_df=pd.DataFrame(stations_data)
weather_df=pd.DataFrame(weather_data)

In [3]:
# look at data to see missing data points.  looks like NaN's in 'prcp' column.
weather_df.count()

station    19550
date       19550
prcp       18103
tobs       19550
dtype: int64

In [4]:
# Quick look at the data
weather_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [5]:
# look at the numbers
weather_df.describe()

,prcp,tobs
count,18103.000000,19550.000000
mean,0.160644,73.097954
std,0.468746,4.523527
min,0.000000,53.000000
25%,0.000000,70.000000
50%,0.010000,73.000000
75%,0.110000,76.000000
max,11.530000,87.000000


In [6]:
# look at data to see missing data points.
stations_df.count()

station      9
name         9
latitude     9
longitude    9
elevation    9
dtype: int64

In [6]:
# full look at the data, appears to be in good shape.
stations_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [7]:
# a pivot table to look at the average rain and temp numbers per station
# will help be evaluate the impact of the NaN's.

nan_eval=weather_df
nan_eval=pd.pivot_table(nan_eval,index=['station'], values=['prcp','tobs'])
nan_eval

,prcp,tobs
station,,
USC00511918,0.047971,71.615968
USC00513117,0.141921,72.689184
USC00514830,0.121058,74.873297
USC00516128,0.429988,70.915008
USC00517948,0.063602,74.684402
USC00518838,0.207222,72.724070
USC00519281,0.212352,71.663781
USC00519397,0.049020,74.553231
USC00519523,0.114961,74.543649


In [8]:
# pivot table to look at the NaN impact of rain volumn on each station.  Since it ranges from 0
# to just over 50% I do not want to drop rows without rain values since I will also be losing
# the temperature (significant impact). From prior pivot table, it appears that if I replace the
# NaN's with 0's I will have less data impact than dropping all of the rows with NaN's.

test=weather_df
test['NaN Count']=test['prcp'].isnull()
test['Count']=1
test=pd.pivot_table(test,index=["station"], values=['Count','NaN Count'],aggfunc=np.sum)
test['percentage']=test['NaN Count']/test['Count']*100
test

,Count,NaN Count,percentage
station,,,
USC00511918,1979,47.0,2.374937
USC00513117,2709,13.0,0.479882
USC00514830,2202,265.0,12.034514
USC00516128,2612,128.0,4.900459
USC00517948,1372,689.0,50.218659
USC00518838,511,169.0,33.072407
USC00519281,2772,0.0,0.000000
USC00519397,2724,39.0,1.431718
USC00519523,2669,97.0,3.634320


In [9]:
# Look at the two files and make sure the weather stations match up.

stat_test1=stations_df['station']
stat_test2=weather_df['station'].unique()
clean= (stat_test1==stat_test2)
clean

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: station, dtype: bool

In [10]:
# Last check is to review the rain and temp data for reasonability 

minrain = weather_df['prcp'].min()
maxrain = weather_df['prcp'].max()
maxtemp = weather_df['tobs'].max()
mintemp = weather_df['tobs'].min()
print('maxrain : ', maxrain, ' | minrain : ',minrain, ' | maxtemp : ', maxtemp, ' | mintemp : ',mintemp)

maxrain :  11.53  | minrain :  0.0  | maxtemp :  87  | mintemp :  53


In [11]:
# replace NaN's with number 0 and save the clean data.  The rain numbers are small (first pivottable)
# and I don't want to trash the temp data, so rather than dropping all the good temp data I'm putting
#  zero in the NaN place since it looks like it won't impact the data as much as the loss of temp.

clean_stations_df=pd.DataFrame(stations_data)
clean_weather_df=pd.DataFrame(weather_data)
clean_weather_df.drop('NaN Count', axis=1, inplace=True)
clean_weather_df.drop('Count', axis=1, inplace=True)
clean_weather_df = clean_weather_df.fillna(0)
clean_weather_df.to_csv("clean_hawaii_measurements.csv", index = False)
clean_stations_df.to_csv("clean_hawaii_stations.csv", index = False)

In [12]:
clean_weather_df

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73
5,USC00519397,2010-01-07,0.06,70
6,USC00519397,2010-01-08,0.00,64
7,USC00519397,2010-01-09,0.00,68
8,USC00519397,2010-01-10,0.00,73
9,USC00519397,2010-01-11,0.01,64
